In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import jenkspy

In [33]:
#!pip install jenkspy --proxy http://rie.proxy.national.agri:8080

In [2]:
def cascade(generateur, nb_iterations):
    fractal = generateur
    for i in range(nb_iterations):
        fractal = np.kron(fractal, generateur)
    return(fractal)

In [3]:
gen1 = np.array([[.1,.1,.1],[.1,.2,.1],[.1,.1,.1]])

In [29]:
test = cascade(gen1,5)

In [30]:
test.shape

(729, 729)

In [ ]:
mpl.use('svg')  

cmap = mpl.cm.RdYlGn_r
#bounds = np.quantile(test, np.arange(0,1,.005))
bounds = jenkspy.jenks_breaks(test.flatten()[np.random.choice(test.flatten().shape[0], 1000, replace=False)], n_classes=5)
norm = mpl.colors.BoundaryNorm(bounds, cmap.N, extend='both')
mpl.cm.ScalarMappable(norm=norm, cmap=cmap)


fig, ax = plt.subplots(figsize=(20,20))
plt.subplots_adjust(bottom=0, right=1, top=1, left=0)

ax.set_axis_off();
ax.imshow(cmap(norm(test)))
plt.savefig("multifractal.svg", format='svg', dpi=300)

In [27]:
mpl.cm.RdYlGn_r

array([1.00e-07, 2.00e-07, 4.00e-07, 8.00e-07, 1.60e-06, 3.20e-06,
       6.40e-06, 1.28e-05])

In [15]:
jenkspy.jenks_breaks(test.flatten()[np.random.choice(test.flatten().shape[0], 1000, replace=False)], n_classes=5)

[1.0000000000000004e-06,
 1.0000000000000004e-06,
 2.0000000000000008e-06,
 4.0000000000000015e-06,
 8.000000000000003e-06,
 1.6000000000000006e-05]